## Initial Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from dateutil import parser

BDM Corpus 2

Bom Dia Mercado (BDM) → xlsx file with BDM articles and more → preprocessing to CSV → export to repository → final dataset

In [ ]:
# load
news_df = pd.read_excel("../data/raw/bdm-corpus-2.xlsx")

# Normalize individual DATE and TIME cells
def parse_datetime_components(date_cell, time_cell):
    try:
        # Coerce both to string and strip spaces
        date_str = str(date_cell).strip()
        time_str = str(time_cell).strip()
        
        # Combine and parse flexibly
        dt = parser.parse(f"{date_str} {time_str}", dayfirst=True)
        return dt.isoformat()
    except Exception:
        return pd.NaT  # mark invalid rows

# Create ISO 8601 Timestamp column
news_df['Timestamp'] = news_df.apply(lambda row: parse_datetime_components(row['DATE'], row['TIME']), axis=1)
news_df['Timestamp'] = pd.to_datetime(news_df['Timestamp'], errors='coerce')

# Drop old columns
news_df.drop(columns=['DATE', 'TIME', 'Index', 'DIRECTION', 'BRER', 'LABEL'], inplace=True)

# Clean newlines in ARTICLE CONTENT and COMMENTS
for col in ['HEADING', 'ARTICLE CONTENT', 'COMMENTS']:
    if col in news_df.columns:
        news_df[col] = news_df[col].astype(str).str.replace(r'[\r\n]+', ' ', regex=True).str.strip()

# Reorder columns
news_df = news_df[['Timestamp'] + [col for col in news_df.columns if col != 'Timestamp']]

# Rename "HEADING" to "Headline" "ARTICLE CONTENT" to "Article" and "COMMENTS" to "Comments"
news_df.rename(columns={
    'HEADING': 'Headline',
    'ARTICLE CONTENT': 'Article',
    'COMMENTS': 'Comments'
}, inplace=True)

# save
news_df.to_csv("../data/interim/bdm-corpus-2/stage-0.csv", index=False, encoding='utf-8-sig')

### Check for invalid rows (rows with no headlines) and drop them

In [ ]:
news_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-0.csv", encoding='utf-8-sig') # reload as csv to ensure correct encoding

invalid_rows = news_df[news_df['Headline'].isna()]
print(f"{len(invalid_rows)} invalid rows found in 'Headline' column.")
display(invalid_rows)

6 invalid rows found in 'Headline' column.


,Timestamp,Headline,Article,Comments
2332,2024-12-16 09:02:00,NaN,NaN,NaN
4299,2025-01-10 12:14:00,NaN,NaN,NaN
4302,2025-01-10 12:14:00,NaN,NaN,NaN
4306,2025-01-10 12:18:00,NaN,NaN,NaN
4367,2025-01-10 15:58:00,NaN,NaN,NaN
4372,2025-01-10 15:59:00,NaN,NaN,NaN


In [ ]:
# drop rows with invalid headlines and resave
news_df = news_df.dropna(subset=['Headline'])
news_df.to_csv("../data/interim/bdm-corpus-2/stage-0.csv", index=False, encoding='utf-8-sig')

## Exchange Rate Preprocessing

Bloomberg → Download USD/BRL exchange rates as excel file → preprocess to CSV → export to repository → final dataset

In [ ]:
# Step 0: Load 
df_usd_brl = pd.read_excel("../data/raw/usd-brl.xlsx")

# Step 1: Clean column names
df_usd_brl.columns = [col.strip() for col in df_usd_brl.columns]
df_usd_brl.rename(columns={"Date": "Raw Timestamp", "Último preço": "USD/BRL"}, inplace=True)

# Step 2: Parse "Raw Timestamp" directly into pandas datetime (no ISO string conversion)
df_usd_brl["Timestamp"] = pd.to_datetime(df_usd_brl["Raw Timestamp"], errors="coerce")

# Step 3: Drop the original column
df_usd_brl.drop(columns=["Raw Timestamp"], inplace=True)

# Step 4: Reorder columns
df_usd_brl = df_usd_brl[["Timestamp", "USD/BRL"]]

# Step 5: Save
df_usd_brl.to_csv("../data/interim/usd-brl.csv", index=False, encoding="utf-8-sig")

In [ ]:
# Load new datasets
bdm_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-0.csv", parse_dates=['Timestamp'])
fx_df = pd.read_csv("../data/interim/usd-brl.csv", parse_dates=['Timestamp'])

In [7]:
# check how many duplicate timestamps there are
num_duplicates = fx_df.duplicated(subset="Timestamp").sum()
print(f"Found {num_duplicates} duplicate timestamps.")

# remove duplicate timestamps (keep first occurrence)
fx_df = fx_df.drop_duplicates(subset="Timestamp", keep="first")
print("Removed duplicate timestamps.")

# set timestamp index and sort
fx_df = fx_df.set_index("Timestamp").sort_index()
print("Set 'Timestamp' as index and sorted chronologically.")

# forward fill missing timestamps to create continuous minute-level series
fx_df = fx_df.resample("1min").ffill()
print("Forward-filled missing minute-level timestamps.")

# verify that the dataframe is now fully continuous
expected_index = pd.date_range(start=fx_df.index.min(), end=fx_df.index.max(), freq="1min")
missing_timestamps = expected_index.difference(fx_df.index)

if missing_timestamps.empty:
    print("Timestamps are now continuous and minute-by-minute. No gaps remain.")
else:
    print(f"{len(missing_timestamps)} missing timestamps still remain:")
    print(missing_timestamps[:10])  # preview first 10 missing

Found 5929 duplicate timestamps.
Removed duplicate timestamps.
Set 'Timestamp' as index and sorted chronologically.
Forward-filled missing minute-level timestamps.
Timestamps are now continuous and minute-by-minute. No gaps remain.


In [ ]:
# save
fx_df.to_csv("../data/interim/usd-brl-continuous.csv", index=True, encoding='utf-8-sig')

## Stage 1

In [ ]:
'''
  - remove all rows following the last timestamp in the interim/bdm-corpus-2/stage-0.csv at 2024-12-30 17:32:00
  - remove article and comments columns from interim/bdm-corpus-2/stage-0.csv
  - merge the USD/BRL values from interim/usd-brl-continuous.csv into interim/bdm-corpus-2/stage-0.csv by matching timestamps
  - compute forward returns based on the prediction horizon t+1 to t+20 minutes
    - positive returns will map to +1, negative returns will map to -1, and no change will map to 0
    - each computed forward return will be stored in a new column named "Forward Return t+X" where X is the number of minutes ahead
  - the function will return a new DataFrame with the merged data and forward returns
'''

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
fx_df = pd.read_csv("../data/interim/usd-brl-continuous.csv", parse_dates=["Timestamp"])
news_df = pd.read_csv("../data/interim/bdm-corpus-2/stage-0.csv", parse_dates=["Timestamp"])

# Restrict news to timestamps before cutoff
last_timestamp = pd.to_datetime("2024-12-30 17:32:00")
news_df = news_df[news_df["Timestamp"] <= last_timestamp]

# Keep only timestamp and headline
news_df = news_df[["Timestamp", "Headline"]]

# --- Compute forward returns within fx_df ---
def compute_forward_returns(df, horizon_minutes=20):
    df = df.copy()
    for i in range(1, horizon_minutes + 1):
        col = f"Forward Return t+{i}"
        delta = df["USD/BRL"].shift(-i) - df["USD/BRL"]
        df[col] = np.sign(delta)  # Maps to -1, 0, or 1
    return df

fx_df = compute_forward_returns(fx_df)

# --- Merge the forward returns into news_df ---
merged_df = pd.merge(news_df, fx_df, on="Timestamp", how="left")

# Report row count
print(f"Number of rows in merged dataset: {len(merged_df)}")

# Save as stage-1 dataset
merged_df.to_csv("../data/interim/bdm-corpus-2/stage-1.csv", index=False, encoding="utf-8-sig")

Number of rows in merged dataset: 3519


## Stage 2

In [ ]:
'''
Stage 2 Dataset Creation Process:
- rename t+5 forward return to ground truth
- remove all t+X forward returns EXCEPT t+5
- remove usd/brl column, as it's not needed for this since the ground truth delta is already calculated

Intraday Trading Sessions: Remove all rows with headlines released outside of market hours (mostly weekends in our data)
    - Mention it's intraday news in the prompt
    - Intraday: news from 5 minutes before market open till 5 minutes before market close
    - For news at end of trading day, the price change is determined by the last trading price (not always t+5), very few instances of this
    - 54 headlines (rows) were removed as they were either neutral or took place on weekends
    - I'd rather keep this a binary classification task and let the "hold" be determined by the trader based on profits or num of trades
'''

In [ ]:
df = pd.read_csv("../data/interim/bdm-corpus-2/stage-1.csv", parse_dates=['Timestamp'])
prices = pd.read_csv("../data/interim/usd-brl.csv", parse_dates=['Timestamp'])

In [ ]:
# --- Filter by TRADING DAYS only ---
valid_dates = set(prices["Timestamp"].dt.date.unique())

def is_trading_day(ts):
    return ts.date() in valid_dates

df = df[df["Timestamp"].apply(is_trading_day)]

# --- Drop USD/BRL column if it exists ---
df.drop(columns=["USD/BRL"], errors="ignore", inplace=True)

# --- Extract and rename t+5 forward return column ---
t5_col = [col for col in df.columns if "t+5" in col and "forward return" in col.lower()]
if t5_col:
    df = df[["Timestamp", "Headline", t5_col[0]]]
    df.rename(columns={t5_col[0]: "Direction"}, inplace=True)

    # --- Map numeric values to direction labels ---
    df["Direction"] = df["Direction"].replace({1.0: "Increase", -1.0: "Decrease", 0.0: "Stable"})

    # --- Drop rows where direction is Stable ---
    df = df[df["Direction"] != "Stable"]

In [ ]:
# --- Rename columns and translate labels to Brazilian Portuguese ---
df.rename(columns={
    "Timestamp": "DataHora",
    "Headline": "Manchete",
    "Direction": "Direção"
}, inplace=True)

df["Direção"] = df["Direção"].replace({
    "Increase": "Aumento",
    "Decrease": "Diminuição"
})

df.to_csv("../data/interim/bdm-corpus-2/stage-2.csv", index=False, encoding="utf-8-sig")

## Stage 3

In [ ]:
'''
- LLaMA-3 preprocessing (keep text natural to preserve syntactic value)
- DO NOT remove stopwords
- DO NOT lemmatize or stem
- DO NOT lowercase
- DO NOT translate or normalize to English
- Preserve accents, diacritics, and original formatting
    
Reason: LLaMA-3 is a base model trained on diverse, natural language

'''

In [ ]:
df = pd.read_csv("../data/interim/bdm-corpus-2/stage-2.csv")

In [25]:
# === Define robust headline cleaner ===
def clean_headline(text: str) -> str:
    text = str(text).strip()

    # Normalize smart quotes and apostrophes
    text = re.sub(r"[“”]", '"', text)
    text = re.sub(r"[‘’]", "'", text)

    # Remove repeated quotes/apostrophes
    text = re.sub(r'"{2,}', '"', text)
    text = re.sub(r"'{2,}", "'", text)

    # Remove leading/trailing quotes (even if multiple)
    text = re.sub(r'^(["\']+)', '', text)
    text = re.sub(r'(["\']+)$', '', text)

    # Remove noisy special character sequences
    text = re.sub(r"[_•√×+÷=<>^~|#*@¬]{2,}", " ", text)
    text = re.sub(r"[_•√×+÷=<>^~|#*@¬]", "", text)

    # Normalize unicode
    text = unicodedata.normalize("NFKC", text)

    # Remove non-printable/unusual characters except Latin-1
    text = re.sub(r"[^\x20-\x7EÀ-ÿ°€¢£¥‰–—…]", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    return text.strip()

In [ ]:
df["Manchete"] = df["Manchete"].apply(clean_headline)
df.to_csv("../data/processed/bdm-corpus-2/stage-3/data.csv", index=False)

### Create train/test sets out of Stage 3 data.csv

In [ ]:
import pandas as pd

df = pd.read_csv("../data/processed/bdm-corpus-2/stage-3/data.csv")  # Adjust path if needed

# Filter by 'Direção'
aumento = df[df["Direção"] == "Aumento"]
diminuicao = df[df["Direção"] == "Diminuição"]

# Sample 100 rows from each category
sample_aumento = aumento.sample(n=200, random_state=42) # reproducible with seed set, same set generated every time
sample_diminuicao = diminuicao.sample(n=200, random_state=42) # same as above

# Combine and shuffle the test set
test_df = pd.concat([sample_aumento, sample_diminuicao])
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Remove test samples from the original DataFrame
to_remove_indices = sample_aumento.index.union(sample_diminuicao.index)
train_df = df.drop(index=to_remove_indices).reset_index(drop=True)

# Save to new CSVs
test_df.to_csv("../data/processed/bdm-corpus-2/stage-3/test.csv", index=False)
train_df.to_csv("../data/processed/bdm-corpus-2/stage-3/train.csv", index=False)